## Module 2: Vector Searching

Vector search leverages machine learning to capture the meaning and context of unstructured data, including text and images, transforming it into a numeric representation. Frequently used for semantic search, vector search finds similar data using approximate nearest neighbor algorithms.

Vector search engines - known as vector databases, semantic, or cosine search - find the nearest neighbors to a given (vectorized) query. Where traditional search relies on mentions of keywords, lexical similarity, and the frequency of word occurrences, vector search engines use distances in the embedding space to represent similarity. Finding related data becomes searching for nearest neighbors of the query.

### Qdrant

Qdrant is an open-source vector search engine, a dedicates solution built in Rust for scalable vector search. 

### Install dependencies

In [9]:
from qdrant_client import QdrantClient, models
import requests
from fastembed import TextEmbedding
import json


In [2]:
qdrant = QdrantClient("http://localhost:6333")

qdrant

### Import the dataset

The dataset has three course types:
- data-engineering-zoomcamp
- machine-learning-zoomcamp
- mlops-zoomcamp

Each course includes a collection of:
- *question:* student's question
- *text:* anwser to student's question
- *section:* course section

In [ ]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'

response = requests.get(docs_url)
documents_raw = response.json()

documents_raw

### Transform documents into embeddings

As we are building a Q&A RAG system, the `question` and `text` fields will be converted to embeddings to find the most relevant answer to a given question.

The `course` and `section` fields can be stored as metadata to provide more context when the someone wants to ask question related to a specific course or a specific course's section.

#### How to choose the embedding model?

It depends on many factors:
- the task, data modality and specifications
- the trade-off between search precision and resource usage (larger embeddings requeri more storage and memory)
- the cost of inference third-party provider

FastEmbed is an optimized embedding solition designed for Qdrant. It supports:
- dense embeddings for text and images (the most common type in vector search)
- sparse embeddings
- multivector embeddings
- rerankers

FastEmbed's integration with Qdrant allows to directly pass text or images to the Qdrant client for embedding.

#### Find the most suitable embedding model

- Use a small embedding model (e.g. 515 dimensions) and suitable for english text.
- Unimodal model once we are not including images in the search, only text.

In [ ]:
EMBEDDING_DIMENSIONALITY = 512

for model in TextEmbedding.list_supported_models():
    if model["dim"] == EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent = 2))

embedding_model = "jinaai/jina-embeddings-v2-small-en"